In [1]:
import importlib
import xarray as xr
import numpy as np
import pandas as pd

from CASutils import filter_utils as filt 
from CASutils import readdata_utils as read
from CASutils import calendar_utils as cal
import sys

importlib.reload(filt)
importlib.reload(read)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

Reading in the data

In [2]:
path="/project/mojave/observations/ERA5_daily/T2m/"
dat = read.read_sfc(path+"*.nc","1979-01-01", "2014-12-31")

Removing Feb 29th

In [10]:
dat = dat.sel(time=~((dat.time.dt.month==2) & (dat.time.dt.day == 29)))

Removing the seasonal cycle (first 4 harmonics of the seasonally varying climatology)

In [13]:
# make an array of the form mon-day for each timestamp.  This is needed for grouping the days of the years properly
# now that we've removed Feb 29th
daystr = xr.DataArray(dat.indexes['time'].strftime('%m-%d'), coords = dat.time.coords, name="daystr")

# Calculate the climatology for each day
datseason = dat.t2m.groupby(daystr).mean('time')

# calculate the seasonal cycle as the first 4 harmonics of the seasonal cycle
t2m4harm = filt.calc_season_nharm(datseason, 4, dimtime=0)

# remove that seasonal cycle
anoms = dat.t2m.groupby(daystr)-t2m4harm

# this is where I'm removing the seasonal mean but you don't have to do that.  Commented out for now.
#djfanoms = cal.group_season_daily(anoms,'DJF')
#djfmean = djfanoms.mean('day')
#djfanoms = djfanoms - djfmean

Calculate the variance

In [16]:
era5var = np.var(djfanoms, axis=(0,1))
era5var = era5var.rename('era5var')

In [17]:
era5var.to_netcdf(path="/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_ERA5.nc")